In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# konlpy 
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# gensim 
!pip install gensim

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data
import konlpy
from konlpy.tag import Mecab
import re
from torchtext import data
from torchtext.vocab import Vectors
from typing import List
import itertools

### 1. 데이터 로드

In [ ]:
#train data, test data path 설정, 확인
data_path_test= "/content/gdrive/My Drive/dataset/BalancedNewsCorpus/BalancedNewsCorpus_test.csv"
data_path_train = "/content/gdrive/My Drive/dataset/BalancedNewsCorpus/BalancedNewsCorpus_train.csv"

In [ ]:
df_train = pd.read_csv(data_path_train, sep = ',')
df_test = pd.read_csv(data_path_test, sep = ',')

### 2. 전처리 함수 정의

In [ ]:
# 불용어 처리 (길호현 (2018). 텍스트마이닝을 위한 한국어 불용어 목록 연구. 우리말글 , 78, 1-25 에 나와있는 불용어 사용. 본 공개코드에선 삭제)
str_stopwords="불용어 입력"
stopwords=str_stopwords.split('\t ')

# tokenizer 함수 정의
mecab=Mecab()
def tokenizer_morphs(string):
    string.replace("…", "")
    string.replace("·", "")
    nouns = mecab.nouns(string) # 명사만 추출
    nouns = [a for a in nouns if (len(a) >1 and a not in stopwords) ]
    return nouns

### 3. Field 정의

In [ ]:
import torch
from torchtext.data import Field, LabelField, TabularDataset

In [ ]:
torch.backends.cudnn.deterministic = True

TEXT = Field(sequential=True, tokenize=tokenizer_morphs, lower=True, batch_first = True) # sequential data, tokenizer라는 토크나이저 사용 / Text 처리하는 방법을 정의
LABEL = LabelField(sequential=False) 

### 4. TabularDataset 생성

In [ ]:
raw_datafields = [("filename", None), 
                 ("date", None), ("NewsPaper", None),
                 ("Topic", LABEL), ("News", TEXT)] 

train_data = TabularDataset(
        path=data_path_train,
        format='csv',
        skip_header=True,
        fields=raw_datafields)

test_data= TabularDataset(
        path=data_path_test,
        format='csv',
        skip_header=True,
        fields=raw_datafields)

### 5. Vocab 생성

In [ ]:
import torchtext.vocab as vocab

TEXT.build_vocab(train_data, min_freq=10)
LABEL.build_vocab(train_data)

In [ ]:
#결과 확인
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fc494fc6730>, {'IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8})


### 6. 사전학습 임베딩 불러오기 
Reference: https://rohit-agrawal.medium.com/using-fine-tuned-gensim-word2vec-embeddings-with-torchtext-and-pytorch-17eea2883cd

In [ ]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm_notebook

#word2vec 형태소 모델의 임베딩 사용
path = '/content/gdrive/My Drive/model/'
Word2Vec_300D_token_model = KeyedVectors.load_word2vec_format(path + 'Word2Vec_300D_token.model', binary=False, encoding='utf-8')

In [ ]:
word2vec_vectors = []

for token, idx in tqdm_notebook(TEXT.vocab.stoi.items()):
    if token in Word2Vec_300D_token_model.wv.vocab.keys(): #사전학습 임베딩 모델에 해당 토큰의 임베딩 값이 있을 경우 그 값을 가져옴
        word2vec_vectors.append(torch.FloatTensor(Word2Vec_300D_token_model[token]))
    else:
        word2vec_vectors.append(torch.randn(300)) #사전학습 임베딩 모델에 임베딩 값이 없을 경우 랜덤으로 설정
        
TEXT.vocab.set_vectors(TEXT.vocab.stoi, word2vec_vectors, 300) #Vocab 각 토큰의 임베딩 값 설정

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  """


### 7. Iterator 정의

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key=lambda x: len(x.News), #데이터를 그룹화하는데 사용하는 함수
 sort_within_batch=False)

### 8 CNN Model
Reference: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
#parameters 설정
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 9
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

#모델에 parameters 입력
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
#모델에 사전학습 임베딩값 가져오기
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 5.2954e-02, -1.1642e-01,  4.9755e-01,  ...,  1.6532e+00,
          1.8065e-01,  8.7345e-01],
        [-9.2371e-01,  1.0091e+00, -1.9192e+00,  ...,  1.1858e+00,
          2.5580e-01, -1.1186e+00],
        [ 9.8296e-02,  6.8959e-02, -9.7626e-02,  ...,  1.9529e-02,
         -4.3655e-02,  5.1375e-02],
        ...,
        [-2.6780e-02, -4.1724e-02,  1.4282e-01,  ..., -1.1069e-02,
         -1.3842e-02,  5.1257e-02],
        [-5.7113e-02, -1.0315e-01,  1.0424e-01,  ..., -6.7090e-02,
          3.6692e-02,  4.1873e-02],
        [-7.6801e-02, -3.9502e-02,  7.4451e-04,  ...,  4.1259e-02,
          2.8716e-03,  5.0831e-02]])

In [ ]:
# <pad> 토큰에 대해 초기 임베딩값을 0으로 설정 
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

### 9. Training 

In [ ]:
#optimizer와 loss function 정의
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.News)
        
        loss = criterion(predictions, batch.Topic)
        
        acc = categorical_accuracy(predictions, batch.Topic)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.News)
            
            loss = criterion(predictions, batch.Topic)
            
            acc = categorical_accuracy(predictions, batch.Topic)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 20

best_train_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if train_loss < best_train_loss:
        best_train_loss = train_loss
    torch.save(model.state_dict(), 'tut5-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 6s
	Train Loss: 1.939 | Train Acc: 33.11%
Epoch: 02 | Epoch Time: 0m 5s
	Train Loss: 1.062 | Train Acc: 65.92%
Epoch: 03 | Epoch Time: 0m 5s
	Train Loss: 0.713 | Train Acc: 76.64%
Epoch: 04 | Epoch Time: 0m 5s
	Train Loss: 0.507 | Train Acc: 83.57%
Epoch: 05 | Epoch Time: 0m 5s
	Train Loss: 0.342 | Train Acc: 89.47%
Epoch: 06 | Epoch Time: 0m 5s
	Train Loss: 0.227 | Train Acc: 93.69%
Epoch: 07 | Epoch Time: 0m 5s
	Train Loss: 0.141 | Train Acc: 96.36%
Epoch: 08 | Epoch Time: 0m 5s
	Train Loss: 0.090 | Train Acc: 98.04%
Epoch: 09 | Epoch Time: 0m 5s
	Train Loss: 0.058 | Train Acc: 98.79%
Epoch: 10 | Epoch Time: 0m 5s
	Train Loss: 0.037 | Train Acc: 99.56%
Epoch: 11 | Epoch Time: 0m 5s
	Train Loss: 0.028 | Train Acc: 99.61%
Epoch: 12 | Epoch Time: 0m 5s
	Train Loss: 0.022 | Train Acc: 99.80%
Epoch: 13 | Epoch Time: 0m 5s
	Train Loss: 0.016 | Train Acc: 99.88%
Epoch: 14 | Epoch Time: 0m 5s
	Train Loss: 0.014 | Train Acc: 99.82%
Epoch: 15 | Epoch Time: 0m 6s
	Tra

### 10. Testing

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.896 | Test Acc: 79.43%


### 11. User Input

####  뉴스 labels
    -  IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8

In [ ]:
def predict_news(model, sentence, min_len=5):
    model.eval()
    tokenized = tokenizer_morphs(sentence)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()